In [1]:
import pandas as pd

data set:

- county
- 2016 vote
- first covid case
- num of cases
- num of deaths
- lockdown?
- lockdown date
- days to lockdown from first case
- pop density
- % of age group

# Corona data

In [19]:
df = pd.read_csv('timeseries-tidy.csv', low_memory=False)

# subset to US
us = df[df['country'] == 'United States']

# subset to deaths
usd = us[us['type'] == 'deaths']

usdc = usd[usd['level'] == 'county']
usdc.to_csv('usdc.csv', index = False)

usds = usd[usd['level'] == 'state']
usds.to_csv('usds.csv', index = False)

# subset to cases
usc = us[us['type'] == 'cases']

uscc = usc[usc['level'] == 'county']
uscc.to_csv('uscc.csv', index = False)

uscs = usc[usc['level'] == 'state']
uscs.to_csv('uscs.csv', index = False)

### First and total cases

In [ ]:
cases = pd.read_csv('./data/uscc.csv')

# total cases by county
totc = cases.loc[:,['state', 'county', 'value']]
totc = totd.groupby(by = ['state', 'county']).sum()

totc.columns = ['cases']

# First case by county
firstc = cases[cases['value'] > 0]
firstc = firstc.loc[:,['state', 'county', 'date', 'value']]

firstc = firstc.groupby(by = ['state', 'county']).min()
firstc.drop('value', axis = 1, inplace = True)
firstc.columns = ['first_case']

### First and total deaths

In [221]:
deaths = pd.read_csv('./data/usdc.csv')

# total deaths by county
totd = deaths.loc[:,['state', 'county', 'value']]
totd = totd.groupby(by = ['state', 'county']).sum()

totd.columns = ['deaths']

# First death by county
firstd = deaths[deaths['value'] > 0]
firstd = firstd.loc[:,['state', 'county', 'date', 'value']]

firstd = firstd.groupby(by = ['state', 'county']).min()
firstd.drop('value', axis = 1, inplace = True)
firstd.columns = ['first_death']

In [222]:
# join cases and deaths
cov = totc.join([firstc, totd, firstd], how = 'left')
cov.to_csv('./data/covid.csv', index = True)

# Lockdown data by country

In [71]:
# read in data from the wiki
wiki = pd.read_html('https://en.wikipedia.org/wiki/Template:2020_coronavirus_pandemic_lockdowns')

# grab the lockdown table
ld = wiki[2]

# rename columns
ld.columns= [col[1] for col in ld.columns]

# export
ld.to_csv('lockdown.csv', index = False)

# Stayhome orders

In [327]:
# import dictionary
import stayhome

#convert to df
stord = stayhome.state_at_home
stord = pd.DataFrame.from_dict(data = stord, orient='index', columns = ['stay_home_date'])
stord.index.rename('state', inplace = True)

# convert to df
cord = stayhome.county_at_home
cord = pd.DataFrame.from_dict(data = cord, orient='index', columns = ['state','stay_home_date'])
cord.set_index('state', append = True, inplace = True)
cord.index.rename(['county', 'state'], inplace=True)

# Population

In [121]:
pop = pd.read_csv('./data/cc-est2018-alldata.csv', encoding = 'ISO-8859-1')

# subset to most recent data
pop = pop[pop['YEAR'] == 11]
# subset to population data
pop = pop.iloc[:,0:8]
# pivot so age group is a column
pp = pop.pivot_table(index = ['STNAME', 'CTYNAME'], columns = 'AGEGRP', values = 'TOT_POP')

# sum all age categories for 65+
pp['65+'] = pp.iloc[:,14:19].sum(axis = 1, numeric_only = True)

# get only population and pop over 65
pp = pp.iloc[:,[0, -1]]

# rename the index
pp.index.rename(['state', 'county'], inplace = True)

# rename the columns
pp.columns = ['pop2018', '65plus2018']

# expor
pp.to_csv('./data/countypop.csv', index = True)

In [122]:
pp.head()

pop2018  65plus2018
state   county                             
Alabama Autauga County    55601        8653
        Baldwin County   218022       44571
        Barbour County    24881        4832
        Bibb County       22400        3690
        Blount County     57840       10548

# Pop Density

In [176]:
# import data
den = pd.read_csv('./data/popdensity.csv')
# subset to columns needed
den = den.loc[:, ['State', 'Area', '2019']]
# rename columns
den.columns = ['state', 'county', 'dens2019']
# export
den.to_csv('popdensity.csv', index=False)

# Election

In [379]:
# import
el = pd.read_excel('./data/2016_election.xlsx')

# get needed columns
el = el[['State', 'County', 'Republicans 2016', 'Democrats 2016', 'Green 2016', 'Libertarians 2016', 'Poverty.Rate.below.federal.poverty.threshold', 'Adult.obesity', ]]

# drop state from the county column
el['county'] = el['county'].str.split(',', n = 1, expand=True)[0]

In [382]:
el

,state,county,r_percent,d_percent,g_percent,l_percent,poverty,obesity
0,Minnesota,Carlton County,45.185226,46.846448,1.290215,3.892796,10.70,0.279
1,Kansas,Morris County,69.704050,22.819315,2.336449,5.140187,11.15,0.330
2,Oklahoma,Okfuskee County,70.963641,23.976608,NaN,5.059751,24.15,0.351
3,Montana,Roosevelt County,49.171902,42.946887,2.569960,4.854369,23.20,0.358
4,New York,Monroe County,40.251559,54.366825,1.548269,3.833348,13.40,0.302
...,...,...,...,...,...,...,...,...
3138,North Carolina,Caswell County,54.759867,43.262007,NaN,1.978126,21.30,0.325
3139,North Dakota,Nelson County,60.507674,31.641086,0.649351,6.670602,9.20,0.308
3140,South Carolina,Greenville County,59.362770,34.663377,0.693381,3.024502,13.95,0.298
3141,Texas,Briscoe County,85.149864,12.397820,0.544959,1.907357,22.10,0.288


# Merge data sources

In [184]:
# join population and population density
ppd = pp.join(den.set_index(['state', 'county']), on = ['state', 'county'], how = 'left')

In [350]:
# join population and covid data
ppdc = ppd.join(cov, how = 'left')

In [351]:
ppdc = ppdc.reset_index()
ppdc = ppdc.merge(stord, how = 'left', on = 'state')

In [385]:
ppdcv = ppdc.set_index(['state', 'county']).join(el.set_index(['state', 'county']), how = 'left')

In [387]:
ppdcv.to_csv('covid_county.csv', index = True)